In [ ]:
modo = input("Digite 1 para modo treino e 2 para modo execução \n")

path = input("Diretório da pasta de treino:\n")
input_eval = int(input("Positivo(1) ou negativo(-1)?\n"))
second_path = input("Diretório da pasta de treino:\n")

other_eval = "negativa" if evaluation == 1 else "positiva"
input_string = "Caminho da pasta " + other_eval + ':\n'
other_path = input(input_string)

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_path_neg = r'D:\Leo\Área de Trabalho\Python\Celero\Celero\Database\train\neg\\'
train_path_pos = r'D:\Leo\Área de Trabalho\Python\Celero\Celero\Database\train\pos\\'
test_path_neg = r'D:\Leo\Área de Trabalho\Python\Celero\Celero\Database\test\neg\\'
test_path_pos = r'D:\Leo\Área de Trabalho\Python\Celero\Celero\Database\test\pos\\'

print(datetime.now())
def read_files(files_path, evaluation, mode):
    text_list = []
    for file in os.listdir(files_path):
        file_review = open(files_path + file,'r', encoding='utf8').read()
        text_list.append(file_review)
    
    df = pd.DataFrame(np.array(text_list),columns=['text'])
    df['evaluation'] = evaluation
    df['mode'] = mode
    return df

train_neg_df = read_files(train_path_neg, -1, "train")
train_pos_df = read_files(train_path_pos, 1, "train")

test_neg_df = read_files(test_path_neg, -1, 'test')
test_pos_df = read_files(test_path_pos, 1, 'test')

df = pd.concat([train_neg_df, train_pos_df, test_neg_df, test_pos_df]).reset_index(drop=True)

2021-02-10 23:52:25.730118


In [8]:
print(datetime.now())
df.text = df.text.astype(str).str.replace(r'<.*?>',' ')
#df.text = df.text.astype(str).str.replace('[^a-zA-Z]',' ').str.lower().str.split()
df.text = df.text.astype(str).str.lower().str.split()

ps = PorterStemmer()
df['stemmed_text'] = df.text.apply(lambda x: [ps.stem(word) for word in x])
df['stemmed_text'] = df['stemmed_text'].str.join(' ')
print(datetime.now())

2021-02-11 00:26:51.418970
2021-02-11 00:29:51.664333


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english", max_features = 2000)
x_train = cv.fit_transform(df[df['mode']=='train']['stemmed_text']).toarray()
x_test = cv.fit_transform(df[df['mode']=='test']['stemmed_text']).toarray()
y_train = df[df['mode']=='train']['evaluation']
y_test = df[df['mode']=='test']['evaluation']

from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5286
[[9057 3443]
 [8342 4158]]
              precision    recall  f1-score   support

          -1       0.52      0.72      0.61     12500
           1       0.55      0.33      0.41     12500

    accuracy                           0.53     25000
   macro avg       0.53      0.53      0.51     25000
weighted avg       0.53      0.53      0.51     25000



from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

inv_vocab = {v: k for k, v in cv.vocabulary_.items()}
vocabulary = [inv_vocab[i] for i in range(len(inv_vocab))]
vocabulary


print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))